In [4]:
import numpy as np

In [26]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size] #create target
    contexts = []

    for idx in range(window_size, len(corpus)-window_size): #target index
        cs = []
        for t in range(-window_size, window_size+1): #context index
            if t == 0:
                continue
            cs.append(corpus[idx+t])
        contexts.append(cs)
    return np.array(contexts), np.array(target)

def convert_one_hot(corpus, vocab_size):
    N = corpus.shape[0] #how many word

    if corpus.ndim == 1: #for example [1, 2, 3, 4, 5, 6]
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1
    elif corpus.ndim == 2: #for example context
        C = corpus.shape[1] #window_size*2
        one_hot = np.zeros((N,C,vocab_size), dtype=np.int32)



In [36]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, 2)



contexts

array([[0, 1, 3, 4],
       [1, 2, 4, 1],
       [2, 3, 1, 5],
       [3, 4, 5, 6]])